# zad 4 Jakub Iliński

In [27]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from typing import List, Tuple, Dict
import torch
import torch.nn.functional as F 
from tqdm import tqdm
import numpy as np

PAPUGA = 'flax-community/papuGaPT2'
DEVICE = "cuda"

tokenizer = AutoTokenizer.from_pretrained(PAPUGA, add_prefix_space=True)
model = AutoModelForCausalLM.from_pretrained(PAPUGA).to(DEVICE)
model.device

device(type='cuda', index=0)

In [28]:
queries: List[str] = []
with open('task4_questions.txt') as file:
    for q in file:
        queries.append(q)
queries.sort()
print(*queries)

Adam Cisowski to postać z której powieści Kornela Makuszyńskiego?
 Adwersarz to sprzymierzeniec czy przeciwnik?
 Afryka leży w większości na półkuli wschodniej czy zachodniej?
 Akcja której komedii rozgrywa się częściowo w Szuflandii?
 Akcyza to podatek pośredni czy bezpośredni?
 Amoniak jest cieczą czy gazem?
 Andantino to tempo szybsze czy wolniejsze od andante?
 Anion posiada przewagę elektronów czy protonów?
 Antoni Patek, pionier produkcji dokładnych zegarków w Szwajcarii, był uczestnikiem którego powstania?
 Aorta wychodzi z serca czy do niego wchodzi?
 Apokalipsa św. Jana to księga dydaktyczna, historyczna czy prorocza?
 Argentyna, Boliwia, Brazylia, Chile – w którym z tych krajów hiszpański nie jest językiem urzędowym?
 Armię którego państwa Napoleon rozbił pod Jeną w 1806 r.?
 Atos i Portos to imiona dwóch muszkieterów – a jak się nazywał trzeci?
 Auksyny to hormony wzrostowe roślin czy zwierząt?
 Automat do gry poruszany za pomocą znajdującej się z boku dźwigni to inaczej „je

### grupy:
1. czy: [tak, nie]
2. ile / Z ilu: [liczba]
3. _ : [normalny prompt]

In [29]:
# full = "ile kotów ma ala"
    
# input_ids = tokenizer(full, return_tensors='pt')['input_ids'].to(DEVICE)

# with torch.no_grad():
#     output = model(input_ids=input_ids)
#     logits, labels = output.logits[:, :-1, :], input_ids[:, 1:]
#     log_probs = log_probs_from_logits(logits, labels)

# logits.size = torch.Size([1, 3, 50257])
# labels.size = torch.Size([1, 3])), 
# input_ids.size() = torch.Size([1, 4])

# for token in logits[0, 0, :]:
#     print(tokenizer.decode(token, skip_special_tokens=True))
    
tokenizer.get_vocab()

inverse = ["" for _ in range(50257)]
for token, idx in tokenizer.get_vocab().items():
    inverse[idx] = token
    
def is_numb(s: str) -> bool:
    try:
        s = float(s)
        return True
    except:
        return False
    
ok = [-np.inf for _ in range(50257)]
for i in range(50257):
    ok[i] = is_numb(inverse[i])
    
ok[:5] = [True for _ in range(5)]
ok[tokenizer.eos_token_id] = True
    
bad_words_ids: List[List[int]] = []
for i in range(50257):
    if ok[i] == False:
        bad_words_ids.append([i])

for i in range(50257):
    if ok[i]: print(inverse[i])
    

<s>
<pad>
</s>
<unk>
<mask>
0
1
2
3
4
5
6
7
8
9
00
01
201
10
20
19
30
50
12
11
000
15
18
200
24
16
14
13
17
25
99
2018
40
60
22
80
23
45
27
26
09
05
21
28
06
03
04
2017
07
08
29
35
02
39
2016
90
70
33
44
2015
48
36
75
100
38
55
46
2013
31
32
34
37
2014
47
49
42
2012
56
43
78
41
65
95
2011
98
67
58
68
97
66
57
88
59
2010
77
52
54
51
53
96
85
83
64
2009
89
69
63
82
87
91
92
72
76
86
94
62
84
500
61
74
79
300
93
73
2008
81
71
9999
400
2020
2007
2006
2019
2005
150
2004
2000
120
600
0000
800
2003
1000
250
110
001
101
2002
700
160
140
2001
125
999999
900
180
130
240
1997
220
112
105
230
210
nan
243
1998
1999
360
123
679
102
350
232
322
128
1945
010
380
111
1939
170
135
104
202
115
365
103
450
320
154
106
107
119
108
822
1200
109
1989
1996
190
261
218
270
280
271
750
116
168
129
1995
239
113
1990
1994
003
124
330
281
175
122
114
131
145
134
264
155
<|endoftext|>


In [30]:
tokenizer.get_vocab()

{'piec': 7547,
 'ĠchÄĻtnych': 14079,
 'ĠprzyjÄĻciem': 37310,
 'ĠzyskÃ³w': 21679,
 'ĠczyniÄħ': 17964,
 'ĠwyglÄħda': 1979,
 'Ġpapa': 41787,
 'Ġkur': 1866,
 'ĠIndii': 26533,
 'ĠKolory': 32376,
 'tj': 19032,
 'Ġeli': 5932,
 'lotÃ³w': 38253,
 'Ġsej': 10826,
 'Ġstadionu': 31819,
 'ĠpoÅ¼arÃ³w': 48881,
 'rowÄħ': 46145,
 'į': 240,
 'Ġestetyka': 48100,
 'powiada': 2097,
 'ĠRegionalny': 37687,
 'Ġmeblami': 42556,
 'jeden': 29875,
 'Ġbudowli': 20565,
 'ĠzaÅĤoÅ¼enia': 8325,
 'ĠministrÃ³w': 44172,
 'czerwone': 44540,
 'ĠspÄĻdza': 11201,
 'ĠksiÄĻ': 2762,
 'Ġzagu': 19538,
 '51': 9364,
 'ĠdotyczÄħcymi': 23364,
 'Ġfil': 1220,
 'Ġnaturalna': 15645,
 'Ġwspo': 1756,
 'Ġpowiedzia': 12019,
 'ĠÅĽwiatowe': 37622,
 'Ġuto': 14275,
 'rocznej': 36664,
 'ĠwypoÅ¼y': 8191,
 'bowych': 16854,
 'vate': 40533,
 'Ġdoradza': 49627,
 'Ġakcji': 3574,
 'ĠskutecznoÅĽci': 15847,
 'KoÅĽciÃ³ÅĤ': 32763,
 'Ca': 6490,
 'ĠbasenÃ³w': 43919,
 'ĠciÄħgÅĤym': 26771,
 'Ġzrelak': 24008,
 'Ġotwarciu': 22099,
 'Stre': 33225,
 'ĠAdministratora

In [31]:
def group_querry(q: str) -> int:
    if q.lower()[:3] == 'czy': return 1
    elif q.lower()[:3] == 'ile' or q.lower()[:5] == 'z ilu': return 2
    else: return 3
    
def log_probs_from_logits(logits, labels):
    logp = F.log_softmax(logits, dim=-1)
    logp_label = torch.gather(logp, 2, labels.unsqueeze(2)).squeeze(-1)
    return logp_label

def get_prob(context: str, label: str) -> float:
    full = context + label
    
    input_ids_full = tokenizer(full, return_tensors='pt')['input_ids'].to(DEVICE)
    input_ids_context = tokenizer(context, return_tensors='pt')['input_ids'].to(DEVICE)
    
    with torch.no_grad():
        output = model(input_ids=input_ids_full)
        logits, labels = output.logits[:, :-1, :], input_ids_full[:, 1:]
        log_probs = log_probs_from_logits(logits, labels)
        
    context_len, full_len = input_ids_context.size(1), input_ids_full.size(1)
    start, end = max(context_len - 1, 0), full_len - 1
    
    label_log_probs = log_probs[:, start:end]
    return float(label_log_probs.mean().item())


def generate_candidates(prompt: str, num: int, max_new_tokens: int, bad_words_ids: List[List[int]]) -> List[str]:
    input_ids = tokenizer(prompt, return_tensors="pt").to(DEVICE)
    
    out = model.generate(
        **input_ids,
        no_repeat_ngram_size=3,
        temperature=0.8,
        max_new_tokens=max_new_tokens,
        do_sample=True,
        num_return_sequences=num,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id,
        bad_words_ids=bad_words_ids,
    )
    
    texts: List[str] = []
    for seq in out:
        full = tokenizer.decode(seq, skip_special_tokens=True)
        new = full[len(prompt):]
        texts.append(new.strip())
        
    return texts

In [32]:
def ans_querry_g1(query: str, mode: str) -> str:
    shots: Dict[str, List[Tuple[str, str]]] = {
        "NO_SHOT": [], 
        "ONE_SHOT": [
            ("Czy pingwin potrafi latać?", "Odpowieź: nie"),
        ],
        "FEW_SHOT": [
            ("Czy pingwin potrafi latać?", "Odpowiedź: nie"),
            ("Czy złoto jest metalem szlachetnym?", "Odpowiedź: tak"),
            ("Czy Sahara leży w Ameryce Południowej?", "Odpowiedź: nie"),
            ("Czy liczba 0 jest liczbą parzystą?", "Odpowiedź: tak"),
            ("Czy Mount Everest znajduje się w Europie?", "Odpowiedź: nie"),
            ("Czy woda wrze w temperaturze 100 stopni Celsjusza przy ciśnieniu normalnym?", "Odpowiedź: tak"),
            ("Czy człowiek ma trzy płuca?", "Odpowiedź: nie"),
            ("Czy Polska ma dostęp do morza?", "Odpowiedź: tak"),
        ],
    }
    
    context = ""
    for q, a in shots[mode]:
        context += f"{q} {a} \n "
    
    prob_yes = get_prob(f"{context} {query} Odpowiedź: ", "tak")
    prob_no = get_prob(f"{context} {query} Odpowiedź: ", "nie")
    return "tak" if prob_yes > prob_no else "nie"

def ans_querry_g2(query: str, mode: str) -> str:
    def pick_best_candidate(candidates: List[str]) -> str:
        ans: List[Tuple[int, str]] = []
        for candidate in candidates:
            score = 0
            for char in candidate:
                if not ord('0') <= ord(char) <= ord('9'):
                    score -= 1
            ans.append((score, candidate))
            
        return sorted(ans, reverse=True)[0][1]

    shots: Dict[str, List[Tuple[str, str]]] = {
        "NO_SHOT": [], 
        "ONE_SHOT": [
            ("Ile dni ma rok przestępny?", "366"),
        ],
        "FEW_SHOT": [
            ("Ile dni ma rok przestępny?", "366"),
            ("Ile kontynentów wyróżnia się na Ziemi?", "7"),
            ("Ile graczy liczy drużyna siatkówki na boisku?", "6"),
            ("Ile boków ma sześciokąt foremny?", "6"),
            ("Ile kilometrów liczy maraton olimpijski?", "42.195"),
            ("Ile bierek ma komplet szachowy na początku partii?", "32"),
            ("Ile minut trwa trzy kwadranse?", "45"),
            ("Ile planet liczy Układ Słoneczny?", "8"),
        ],
    }
    
    context = ""
    for q, a in shots[mode]:
        context += f"{q} {a} \n "
        
    candidates: List[str] = generate_candidates(f"{context}{query}", 7, 2, bad_words_ids)
    return pick_best_candidate(candidates)

def ans_querry_g3(query: str, mode: str) -> str:
    def pick_best_candidate(candidates: List[str]) -> str:
        ans: List[Tuple[int, str]] = []
        for candidate in candidates:
            score = 0
            for char in candidate:
                if not ord('a') <= ord(char) <= ord('z') and not char in ['ą', 'ę', 'ź', 'ć', 'ś']:
                    score -= 1
            ans.append((score, candidate))
            
        return max(ans)[1]
    
    shots: Dict[str, List[Tuple[str, str]]] = {
        "NO_SHOT": [], 
        "ONE_SHOT": [
            ("Jakie miasto leży najbliżej Morskiego Oka?", "Zakopane"),
        ],
        "FEW_SHOT": [
            ("Jakie miasto leży najbliżej Morskiego Oka?", "Zakopane"),
            ("Jak nazywa się najstarszy polski uniwersytet?", "Uniwersytet Jagielloński"),
            ("Jakie miasto leży nad rzeką Brdą?", "Bydgoszcz"),
            ("Które polskie miasto nazywane jest 'stolicą Tatr'?", "Zakopane"),
            ("Jak nazywa się park narodowy położony w Tatrach?", "Tatrzański Park Narodowy"),
            ("Jakie miasto nazywane jest 'Perłą Renesansu'?", "Zamość"),
            ("Jak nazywa się najdłuższy tunel w Polsce?", "Tunel Zakopianka"),
            ("Jak nazywa się najgłębsze jezioro w Polsce?", "Hańcza"),
        ],
    }
    
    context = ""
    for q, a in shots[mode]:
        context += f"{q} {a} \n "
        
    candidates: List[str] = generate_candidates(f"{context}{query}", 7, 5, None)
    return pick_best_candidate(candidates)

In [33]:
def ans_querry(q: str, mode) -> str:
    match group_querry(q):
        case 1: return ans_querry_g1(q, mode)
        case 2: return ans_querry_g2(q, mode)
        case _: return ans_querry_g3(q, mode)

MODE = "FEW_SHOT"           # "NO_SHOT", "ONE_SHOT", "FEW_SHOT"
with open('task4_questions.txt', 'r') as file, open('found_answers.txt', 'w') as ansf:
    for line in tqdm(file):
        ans = ans_querry(line, MODE)
        ansf.write(f"{ans}\n")

2500it [02:47, 14.92it/s]


In [34]:
# with open('task4_questions.txt') as q, open('found_answers.txt') as a:
#     for lq, la in zip(q, a):
#         print(group_querry(lq), " : ", lq, la)
        
nums, ifff = 0, 0 
with open('task4_questions.txt') as q, open('found_answers.txt') as my_a, open('task4_answers.txt') as true_a:
    for question, my_ans, correct_ans in zip(q, my_a, true_a):
        if group_querry(question) == 2: 
            try:
                if int(correct_ans) == int(my_ans): nums += 1
            except:
                continue
        if group_querry(question) == 1:
            if my_ans == correct_ans: ifff += 1

nums, ifff

(0, 58)

In [ ]:
import editdistance
import sys
import re

group_scores = [0, 0, 0]

def scaled_editdist(ans, cor):
    ans = ans.lower()
    cor = cor.lower()
    
    return editdistance.eval(ans, cor) / len(cor)
    
def single_match(a, c):
    if c.isdecimal():
        return a == c
    return scaled_editdist(a, c) < 0.5
        
def match(ans, cor):
    return any(single_match(ans, c) for c in cor)
        
found_answers = []
correct_answers = []

for x in open('task4_answers.txt'):
    x = x.strip()
    correct_answers.append(x.lower().split('\t'))
    
for x in open('found_answers.txt'):    
    x = x.strip()
    found_answers.append(x.lower())
    
N = len(correct_answers)
score = 0.0

with open('task4_questions.txt') as questions_file:
    for ans, cor, q in zip(found_answers, correct_answers, questions_file):    
        ans = 
        if match(ans, cor):
            score += 1
            
            group_scores[group_querry(q)] += 1
        
print ('TOTAL SCORE:', score)
group_scores

TOTAL SCORE: 58.0


[0, 58, 0]